# 前言 - 学习本项目你可以获得什么
- 入门百度智能云千帆AppBuilder，搭建一个工作流Agent应用
- 使用AppBuilder-SDK进行工作流Agent对话
- 了解如何使用AppBuilder-SDK简化多轮对话的开发

## 1. 项目背景

### 1.1、 什么是AppBuilder
[百度智能云千帆AppBuilder](https://appbuilder.cloud.baidu.com/)（以下简称AppBuilder）是基于大模型搭建AI原生应用的工作台，旨在降低AI原生应用的开发门槛，赋能开发者和企业快速实现应用搭建。

平台提供了RAG（检索增强生成）、Agent（智能体）等应用框架，内置了文档问答、表格问答、多轮对话、生成创作等多种应用组件，还包括百度搜索和百度地图等特色组件，以及文本处理、图像处理和语音处理等传统AI组件，支持零代码、低代码、全代码三种开发方式，满足不同开发能力的开发者和企业的场景需求。

### 1.2、 什么是AppBuilder-SDK

[百度智能云千帆AppBuilder-SDK](https://github.com/baidubce/app-builder)(以下简称AB-SDK)，百度智能云千帆AppBuilder-SDK是百度智能云千帆AppBuilder面向AI原生应用开发者提供的一站式开发平台的客户端SDK。

<img src="https://chengmo-dev1.bj.bcebos.com/page2.png" alt="drawing" width="1000"/>

# 2. 创建工作流Agent应用并对话

## 2.1 什么是工作流Agent?
通过工作流编排的形式还原业务流程，每轮对话均严格按照工作流执行，提高了AI应用的可控性，并可编排出复杂业务流程，适用于客服、营销、生成、办公等高可控及高复杂度等场景。

**工作流Agent无需设置角色人设，通过工作流编排的形式实现应用功能。**

用户的所有对话均会触发此工作流处理，适用于严格按照流程执行的任务，例如：

* 客服对话智能体，判断终端用户的意图后严格按照任务分支自动执行，无需大模型思考选择。
* MBTI小助手，根据开发者编排的问题逐个提问，严格按照任务分支和结果执行后续的流程。

工作流Agent支持用户配置工作流完成整个应用的对话过程，通过开始节点的Rawquery传入首轮对话，利用信息收集节点可以进行一组工作流中的多轮对话，最后以结束节点作为整个工作流结束的标志。开始节点和结束节点之前的完整流程构成一组工作流，每个信息收集节点都可以支持终端用户和应用的一次交互。

## 2.2 创建工作流Agent应用-飞行客服小助手

参考产品文档[飞行客服小助手](https://cloud.baidu.com/doc/AppBuilder/s/cm38k8nqr)创建示例应用。通过阅读本篇最佳实践，读者可以深度理解问答节点、信息处理节点、全局跳转节点的功能点和使用方式，搭建自己的工作流agent。对话效果如下：

<p float="left">
  <img src="https://bce.bdstatic.com/doc/ai-cloud-share/AppBuilder/image_f11055c.png" width="33%" />
  <img src="https://bce.bdstatic.com/doc/ai-cloud-share/AppBuilder/image_a680834.png" width="33%" /> 
  <img src="https://bce.bdstatic.com/doc/ai-cloud-share/AppBuilder/image_b602fef.png" width="33%" />
</p>

## 2.3 使用AppBuilder-SDK进行对话
### 2.3.1 方式一：使用SDK直接对话（不推荐）

In [ ]:
import appbuilder
import os
from appbuilder.core.console.appbuilder_client import data_class

# 请前往千帆AppBuilder官网创建密钥，流程详见：https://cloud.baidu.com/doc/AppBuilder/s/Olq6grrt6#1%E3%80%81%E5%88%9B%E5%BB%BA%E5%AF%86%E9%92%A5
# 设置环境变量
os.environ["APPBUILDER_TOKEN"] = "..."
appbuilder.logger.setLoglevel("ERROR")

# 飞行客服小助手的应用id
app_id = "..."
# 初始化智能体
client = appbuilder.AppBuilderClient(app_id)
# 创建会话
conversation_id = client.create_conversation()
interrupt_ids = []

msg = client.run(conversation_id, "查天气", stream=True)
interrupt_event_id = None
for ans in msg.content:
    for event in ans.events:
        if event.content_type == "publish_message":
            print(event.detail.get("message"))
        if event.content_type == "chatflow_interrupt":
            interrupt_event_id = event.detail.get("interrupt_event_id")
            break
interrupt_ids.append(interrupt_event_id)

msg2 = client.run(
    conversation_id,
    "查航班",
    stream=True,
    action=data_class.Action.create_resume_action(interrupt_event_id),
)
interrupt_event_id = None
for ans in msg2.content:
    for event in ans.events:
        if event.content_type == "publish_message":
            print(event.detail.get("message"))
        if event.content_type == "chatflow_interrupt":
            interrupt_event_id = event.detail.get("interrupt_event_id")
            break
        interrupt_ids.append(interrupt_event_id)

msg3 = client.run(
    conversation_id=conversation_id,
    query="CA1234",
    stream=True,
    action=data_class.Action.create_resume_action(interrupt_ids.pop()),
)
interrupt_event_id = None
for ans in msg3.content:
    for event in ans.events:
        if event.content_type == "text":
            print(event.detail.get("text"))
        if event.content_type == "chatflow_interrupt":
            interrupt_event_id = event.detail.get("interrupt_event_id")
            break
interrupt_ids.append(interrupt_event_id)

msg4 = client.run(
    conversation_id=conversation_id,
    query="北京的",
    stream=True,
    action=data_class.Action.create_resume_action(interrupt_ids.pop()),
)
has_multiple_dialog_event = False
for ans in msg4.content:
    for event in ans.events:
        if event.content_type == "text":
            print(event.detail.get("text"))
        if event.content_type == "multiple_dialog_event":
            has_multiple_dialog_event = True
            break

### 2.3.2 方式二 实现自己的EventHandler，更方便地进行对话(推荐)
直接调用处理较为繁琐。SDK提供了使用AppBuilderEventHandler简化tool_call操作的功能。**继承AppBuilderEventHandler类，并实现针对各类型event的处理方法。**

In [ ]:
import os
import appbuilder
from appbuilder.core.console.appbuilder_client.event_handler import (
    AppBuilderEventHandler,
)


class MyEventHandler(AppBuilderEventHandler):
    def __init__(self):
        super().__init__()
        self.interrupt_ids = []

    def handle_content_type(self, run_context, run_response):
        interrupt_event_id = None
        event = run_response.events[-1]
        if event.content_type == "chatflow_interrupt":
            interrupt_event_id = event.detail.get("interrupt_event_id")
        if interrupt_event_id is not None:
            self.interrupt_ids.append(interrupt_event_id)

    def _create_action(self):
        if len(self.interrupt_ids) == 0:
            return None
        event_id = self.interrupt_ids.pop()
        return {
            "action_type": "resume",
            "parameters": {"interrupt_event": {"id": event_id, "type": "chat"}},
        }

    def run(self, query=None):
        super().new_dialog(
            query=query,
            action=self._create_action(),
        )


def main():
    # 请前往千帆AppBuilder官网创建密钥，流程详见：https://cloud.baidu.com/doc/AppBuilder/s/Olq6grrt6#1%E3%80%81%E5%88%9B%E5%BB%BA%E5%AF%86%E9%92%A5
    # 设置环境变量
    os.environ["APPBUILDER_TOKEN"] = "..."
    appbuilder.logger.setLoglevel("DEBUG")

    # 飞行客服小助手的应用id
    app_id = "..."
    # 初始化智能体
    client = appbuilder.AppBuilderClient(app_id)
    conversation_id = client.create_conversation()

    event_handler = MyEventHandler()
    event_handler.init(
        appbuilder_client=client,
        conversation_id=conversation_id,
        stream=True,
        query="查天气",
    )
    for data in event_handler:
        pass
    event_handler.run(
        query="查航班",
    )
    for data in event_handler:
        pass
    event_handler.run(
        query="CA1234",
    )
    for data in event_handler:
        pass
    event_handler.run(
        query="北京的",
    )
    for data in event_handler:
        pass


if __name__ == "__main__":
    main()

### 2.3.3 方式三、实现自己的EventHandler，更方便地进行多轮对话（推荐）
SDK提供了run_multiple_dialog_with_handler方法来进行多轮对话，按需传入iterable对象(queries、actions等参数)，即可一次调用完成多轮对话。

In [ ]:
import os
import appbuilder
from appbuilder.core.console.appbuilder_client.event_handler import (
    AppBuilderEventHandler,
)


class MyEventHandler(AppBuilderEventHandler):
    def __init__(self):
        super().__init__()
        self.interrupt_ids = []

    def handle_content_type(self, run_context, run_response):
        interrupt_event_id = None
        event = run_response.events[-1]
        if event.content_type == "chatflow_interrupt":
            interrupt_event_id = event.detail.get("interrupt_event_id")
        if interrupt_event_id is not None:
            self.interrupt_ids.append(interrupt_event_id)

    def _create_action(self):
        if len(self.interrupt_ids) == 0:
            return None
        event_id = self.interrupt_ids.pop()
        return {
            "action_type": "resume",
            "parameters": {"interrupt_event": {"id": event_id, "type": "chat"}},
        }

    def gen_action(self):
        while True:
            yield self._create_action()


def main():
    # 请前往千帆AppBuilder官网创建密钥，流程详见：https://cloud.baidu.com/doc/AppBuilder/s/Olq6grrt6#1%E3%80%81%E5%88%9B%E5%BB%BA%E5%AF%86%E9%92%A5
    # 设置环境变量
    os.environ["APPBUILDER_TOKEN"] = "..."
    appbuilder.logger.setLoglevel("DEBUG")

    # 飞行客服小助手的应用id
    app_id = "..."
    # 初始化智能体
    client = appbuilder.AppBuilderClient(app_id)
    conversation_id = client.create_conversation()

    queries = ["查天气", "查航班", "CA1234", "北京的"]
    event_handler = MyEventHandler()
    event_handler = client.run_multiple_dialog_with_handler(
        conversation_id=conversation_id,
        queries=queries,
        event_handler=event_handler,
        stream=True,
        actions=event_handler.gen_action(),
    )
    for data in event_handler:
        for ans in data:
            pass


if __name__ == "__main__":
    main()

# 3、项目总结

本项目最终完成了一个工作流Agent应用的创建，并使用Appbuilder-SDK进行对话功能。

希望您可以不吝`Star`，给`AppBuilder-SDK`一些鼓励，期待您的`PR`，一起共建AIAgent生态。

Github地址：https://github.com/baidubce/app-builder

<img src="https://chengmo-dev1.bj.bcebos.com/page10.png" alt="drawing" width="1000"/>

最后，您也可以进入`AppBuilder-SDK`的WX交流群，和大家一起交流AppBuilder使用及开发心得。

<img src="https://chengmo-dev1.bj.bcebos.com/wechat_group.png" alt="drawing" width="1000"/>